In [1]:
import numpy as np
from pydrake.all import (DiagramBuilder, Integrator, JacobianWrtVariable,
                         LeafSystem, MeshcatVisualizer, Simulator,
                         StartMeshcat)

from manipulation import FindResource, running_as_notebook
from manipulation.scenarios import MakeManipulationStation


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
class PINV_Controller(LeafSystem): 
    
    def __init__(self, plant):
        
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName('iiwa')
        self._G = plant.GetBodyByName('body').body_frame()
        self._W = plant.world_frame()
        
        # question: why we provide a velocity in output?
        self.DeclareVectorInputPort('position', 7)
        #                                           function to calc output
        self.DeclareVectorOutputPort('velocity', 7, self.CalcOutput)
        
        
    def CalcOutput(self, context, output): 
        q = self.get_input_port().Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        
        Jac_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot, 
            self._G, 
            [0, 0, 0], # pos vector origin
            self._W,
            self._W
        )
        Jac_G = Jac_G[:, 0:7] # remove gripper col
        
        
        # questo dovrebbe essere l'errore per raggiungere
        # una particolare posizione : mantenendolo costante otteniamo 
        # un movimento costante
         
        Vel_G_desired = np.array([0, # ROTATION VEL 
                                 -1,
                                  0, 
                            #------------------------
                                  0,  #TRANSLATION
                                 -0.05, 
                                 -0.01])
        
        #q_dot_required = J^-1 * (error) = J^-1 * vel_desired_To_apply
        
        #main limitation: non invertible jac ---> crash
        v = np.linalg.pinv(Jac_G).dot(Vel_G_desired)
        output.SetFromVector(v)

In [5]:
def jac_controller_example(): 
    
    builder = DiagramBuilder()
    station = builder.AddSystem(
        MakeManipulationStation(
            filename=FindResource('models/iiwa_and_wsg.dmd.yaml')            
        )
    )
    plant = station.GetSubsystemByName('plant')
    
    controller = builder.AddSystem(PINV_Controller(plant))
    integrator = builder.AddSystem(Integrator(7))
    
    
    builder.Connect(controller.get_output_port(), integrator.get_input_port())
    # why the input is the position? 
    builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa_position"))
    builder.Connect(station.GetOutputPort("iiwa_position_measured"), controller.get_input_port())
    
    meshcat.Delete()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort('query_object'), meshcat
    )
    
    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    station_context = station.GetMyContextFromRoot(context)
    
    
    station.GetInputPort('iiwa_feedforward_torque').FixValue(station_context, np.zeros([7, 1]))
    station.GetInputPort('wsg_position').FixValue(station_context, [0.1])
    
    integrator.set_integral_value(
        integrator.GetMyContextFromRoot(context), 
        plant.GetPositions(plant.GetMyContextFromRoot(context), 
                          plant.GetModelInstanceByName('iiwa'))
    )
    
    visualizer.StartRecording(False)
    simulator.AdvanceTo(5.0)
    visualizer.PublishRecording()
    

jac_controller_example()